In [0]:
secret=dbutils.secrets.get('databricklogin','databricklogin')

In [0]:
application_id ="88dec17f-62f0-4cab-82b7-b9022c2cc21c"
service_credential=secret
directory_id="037e0179-cd0e-4201-951c-167b7554d77a"
spark.conf.set("fs.azure.account.auth.type.datastoragenew12.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.datastoragenew12.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.datastoragenew12.dfs.core.windows.net", application_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.datastoragenew12.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.datastoragenew12.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
display(dbutils.fs.ls("abfss://proccesed@datastoragenew12.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://proccesed@datastoragenew12.dfs.core.windows.net/Address/,Address/,0,1732923206000
abfss://proccesed@datastoragenew12.dfs.core.windows.net/Customer/,Customer/,0,1732923982000
abfss://proccesed@datastoragenew12.dfs.core.windows.net/CustomerAddress/,CustomerAddress/,0,1732923986000
abfss://proccesed@datastoragenew12.dfs.core.windows.net/Product/,Product/,0,1732923990000


In [0]:
df_customer_address = spark.read.format("delta").load("abfss://proccesed@datastoragenew12.dfs.core.windows.net/Address")
df_customer_phone = spark.read.format("delta").load("abfss://proccesed@datastoragenew12.dfs.core.windows.net/Customer")
df_customer_address.show(5)
df_customer_phone.show(5)


+---------+-------------------+------------+--------+-------------+-------------+----------+--------------------+----------+
|AddressID|       AddressLine1|AddressLine2|    City|StateProvince|CountryRegion|PostalCode|             rowguid|      Date|
+---------+-------------------+------------+--------+-------------+-------------+----------+--------------------+----------+
|        9|  8713 Yosemite Ct.|        NULL| Bothell|   Washington|United States|     98011|268af621-76d7-4c7...|2006-07-01|
|       11|1318 Lasalle Street|        NULL| Bothell|   Washington|United States|     98011|981b3303-aca2-49c...|2007-04-01|
|       25|   9178 Jumping St.|        NULL|  Dallas|        Texas|United States|     75201|c8df3bd9-48f0-465...|2006-09-01|
|       28|   9228 Via Del Sol|        NULL| Phoenix|      Arizona|United States|     85004|12ae5ee1-fc3e-468...|2005-09-01|
|       32|  26910 Indela Road|        NULL|Montreal|       Quebec|       Canada|   H1Y 2H5|84a95f62-3ae8-4e7...|2006-08-01|


In [0]:
#check the customers in the address table

In [0]:
df_customer_address = df_customer_address.withColumn("CustomerID", df_customer_address["AddressID"])
# Perform the join
df_customer_details = df_customer_address.join(df_customer_phone, "CustomerID")
df_customer_details = df_customer_details.select("CustomerID", "AddressLine1", "Phone","FirstName", "LastName")

display(df_customer_details)

In [0]:
df_customer_details.write.format("delta").mode("overwrite").save("abfss://gold@datastoragenew12.dfs.core.windows.net/customer_details")